In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from datetime import datetime
import pandas as pd

#spark = SparkSession.builder.master("local[1]").appName("part2").config("spark.executor.instances", "3").getOrCreate()
#spark.conf.set("spark.sql.execution.arrow.enabled", "true")

spark = SparkSession.builder.config("spark.jars", "/usr/share/java/mysql-connector-j-8.0.31.jar") \
    .master("local").appName("PySpark_MySQL_test").getOrCreate()

23/12/07 04:28:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
df = spark.read.format("csv").option("header", "true").load("/home/ubuntu/charts.csv")

In [6]:
df.show()

+--------------------+----+----------+--------------------+--------------------+---------+------+-------------+-------+
|               title|rank|      date|              artist|                 url|   region| chart|        trend|streams|
+--------------------+----+----------+--------------------+--------------------+---------+------+-------------+-------+
|Chantaje (feat. M...|   1|2017-01-01|             Shakira|https://open.spot...|Argentina|top200|SAME_POSITION| 253019|
|Vente Pa' Ca (fea...|   2|2017-01-01|        Ricky Martin|https://open.spot...|Argentina|top200|      MOVE_UP| 223988|
|Reggaetón Lento (...|   3|2017-01-01|                CNCO|https://open.spot...|Argentina|top200|    MOVE_DOWN| 210943|
|              Safari|   4|2017-01-01|J Balvin, Pharrel...|https://open.spot...|Argentina|top200|SAME_POSITION| 173865|
|         Shaky Shaky|   5|2017-01-01|        Daddy Yankee|https://open.spot...|Argentina|top200|      MOVE_UP| 153956|
|         Traicionera|   6|2017-01-01|  

pre-processing

In [7]:
# dropping missing or null values
df = df.dropna()

# create "newdate" column as a date datatype since current date column is a string
# want date datatype so that we can use the sql date functions
df = df.withColumnRenamed("date", "olddate")
df = df.withColumn('date', to_date('olddate'))
df = df.drop('olddate')

df.show()
# count of artist whose songs move up in the chart during a specific year order by count desc
# x axis year, y artist move up value

# for each artist group by artist max streams of an artist (by song?)

+--------------------+----+--------------------+--------------------+---------+------+-------------+-------+----------+
|               title|rank|              artist|                 url|   region| chart|        trend|streams|      date|
+--------------------+----+--------------------+--------------------+---------+------+-------------+-------+----------+
|Chantaje (feat. M...|   1|             Shakira|https://open.spot...|Argentina|top200|SAME_POSITION| 253019|2017-01-01|
|Vente Pa' Ca (fea...|   2|        Ricky Martin|https://open.spot...|Argentina|top200|      MOVE_UP| 223988|2017-01-01|
|Reggaetón Lento (...|   3|                CNCO|https://open.spot...|Argentina|top200|    MOVE_DOWN| 210943|2017-01-01|
|              Safari|   4|J Balvin, Pharrel...|https://open.spot...|Argentina|top200|SAME_POSITION| 173865|2017-01-01|
|         Shaky Shaky|   5|        Daddy Yankee|https://open.spot...|Argentina|top200|      MOVE_UP| 153956|2017-01-01|
|         Traicionera|   6|     Sebastia

In [14]:
# sentiment analysis of song lyrics
df_s = spark.read.format("csv").option("header", "true").load("/home/ubuntu/sentimentFixed.csv")

In [15]:
df_s = df_s.filter(df_s.Sentiment.isNotNull())
df_s = df_s.drop('_c0')
df_s = df_s.distinct()
df_s.show()
df_s.count()
df_s = df_s.drop('pos')
df_s = df_s.drop('neu')
df_s = df_s.drop('neg')
df_s.show()

+--------------------+--------------------+------------------+-----------------+------------------+---------+
|                Song|              Artist|               pos|              neu|               neg|Sentiment|
+--------------------+--------------------+------------------+-----------------+------------------+---------+
|       Carnavalintro|               Chano|               0.0|94.73684210526315| 5.263157894736842| Negative|
|         Sex on Fire|       Kings of Leon|4.3478260869565215|95.65217391304348|               0.0| Positive|
|Ya Me Enteré (fea...|                Reik|               0.0|            98.75|              1.25| Negative|
|Como Faz Com Ela ...|    Marília Mendonça|16.071428571428573|83.92857142857143|               0.0| Positive|
|Paluu tulevaisuuteen|                 JVG|               0.0|            100.0|               0.0|  Neutral|
|       We Found Love|Rihanna, Calvin H...|             3.125|           96.875|               0.0| Positive|
|Don Medel

In [8]:
df_ly = spark.read.format("csv").option("header", "true").load("/home/ubuntu/cleanedTestAuto.csv")
df_ly.show()

+--------------------+--------------------+--------------------+
|              Artist|                Song|              Lyrics|
+--------------------+--------------------+--------------------+
|             Shakira|Chantaje (feat. M...|"""2 Contributors...|
|        Ricky Martin|Vente Pa' Ca (fea...|"'3 ContributorsN...|
|                CNCO|Reggaet�n Lento (...|"""25 Contributor...|
|J Balvin, Pharrel...|              Safari|"""39 Contributor...|
|        Daddy Yankee|         Shaky Shaky|"""22 Contributor...|
|     Sebastian Yatra|         Traicionera|'11 ContributorsT...|
|              Rombai|Cuando Se Pone a ...|'3 ContributorsCu...|
|       Zion & Lennox|Otra vez (feat. J...|"""2 Contributors...|
|Carlos Vives, Sha...|        La Bicicleta|"""50 Contributor...|
|               Ozuna|Dile Que Tu Me Qu...|"'33 Contributors...|
|Chino & Nacho, Da...|  Andas En Mi Cabeza|"'10 Contributors...|
|              Thalia|Desde Esa Noche (...|'1 ContributorTha...|
|              Maluma|   

mysql

In [2]:
import mysql.connector

db_connection = mysql.connector.connect(user="hpham", password="halpal")
db_cursor = db_connection.cursor()
db_cursor.execute("USE cs179g;")

In [5]:
temp = spark.read.format("jdbc").option("driver","com.mysql.jdbc.Driver").option("url", "jdbc:mysql://localhost:3306/cs179g").option("dbtable", "ArtistByRegion").option("user", "hpham").option("password", "halpal").load()

temp.show()

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


+---------+-------------------+----------+
|   region|             artist|song_count|
+---------+-------------------+----------+
|Argentina|             Maluma|      5673|
|Argentina|       Paulo Londra|      5630|
|Argentina|         Ed Sheeran|      5397|
|Australia|         Ed Sheeran|     13194|
|Australia|        Post Malone|      7907|
|Australia|      Billie Eilish|      7141|
|  Austria|         Ed Sheeran|      5431|
|  Austria|        Capital Bra|      4695|
|  Austria|      Billie Eilish|      4275|
|  Belgium|         Ed Sheeran|      8567|
|  Belgium|      Billie Eilish|      6366|
|  Belgium|       XXXTENTACION|      5604|
|  Bolivia|          Bad Bunny|      8703|
|  Bolivia|             Maluma|      6296|
|  Bolivia|    Christian Nodal|      4676|
|   Brazil|   Marília Mendonça|     11059|
|   Brazil|Zé Neto & Cristiano|      9385|
|   Brazil| Henrique & Juliano|      8914|
| Bulgaria|          Pop Smoke|      3265|
| Bulgaria|       Travis Scott|      2760|
+---------+

In [6]:
db_cursor.execute("SELECT * FROM Seasons WHERE season='spring';")
# Fetch all the results
results = db_cursor.fetchall()

# Display the results
for row in results:
    print(row)

(38, 'Drake', 'spring')
(38, 'Lil Uzi Vert', 'spring')
(33, 'Taylor Swift', 'spring')
(32, 'BTS', 'spring')
(31, 'Ed Sheeran', 'spring')
(29, 'Juice WRLD', 'spring')
(27, 'J. Cole', 'spring')
(27, 'The Weeknd', 'spring')
(26, 'Post Malone', 'spring')
(24, 'Logic', 'spring')


ArtistByRegion - the top 3 most popular artists per region (per region chart, the # of times their song showed up on the top200 chart over all years) 

In [7]:
df.createOrReplaceTempView("songs")
temp = spark.sql("""SELECT * FROM songs WHERE chart=='top200'""")
temp.createOrReplaceTempView('top200')

In [14]:
cloud = spark.sql("SELECT region,artist, COUNT(*) AS song_count, ROW_NUMBER() OVER(PARTITION BY region ORDER BY COUNT(*) DESC) AS artist_rank FROM top200 GROUP BY region, artist")
cloud.createOrReplaceTempView("clouds")
all_regions = spark.sql("SELECT region, artist, song_count FROM clouds WHERE artist_rank <= 3")
all_regions.show()

+---------+-------------------+----------+
|   region|             artist|song_count|
+---------+-------------------+----------+
|Argentina|             Maluma|      5673|
|Argentina|       Paulo Londra|      5630|
|Argentina|         Ed Sheeran|      5397|
|Australia|         Ed Sheeran|     13194|
|Australia|        Post Malone|      7907|
|Australia|      Billie Eilish|      7141|
|  Austria|         Ed Sheeran|      5431|
|  Austria|        Capital Bra|      4695|
|  Austria|      Billie Eilish|      4275|
|  Belgium|         Ed Sheeran|      8567|
|  Belgium|      Billie Eilish|      6366|
|  Belgium|       XXXTENTACION|      5604|
|  Bolivia|          Bad Bunny|      8703|
|  Bolivia|             Maluma|      6296|
|  Bolivia|    Christian Nodal|      4676|
|   Brazil|   Marília Mendonça|     11059|
|   Brazil|Zé Neto & Cristiano|      9385|
|   Brazil| Henrique & Juliano|      8914|
| Bulgaria|          Pop Smoke|      3265|
| Bulgaria|       Travis Scott|      2760|
+---------+

In [9]:
all_regions.write.format("jdbc").option("driver","com.mysql.jdbc.Driver").option("url", "jdbc:mysql://localhost:3306/cs179g").option("dbtable", "ArtistByRegion").option("user", "hpham").option("password", "halpal").save()

NameError: name 'all_regions' is not defined

Seasons - top 10 most popular artists per season (based on global charts)

In [17]:
# create temp view and filter for only the global top 200 charts
df.createOrReplaceTempView('df_dates')
temp = spark.sql("""
  SELECT EXTRACT(MONTH FROM date) as month, artist, title, region, rank FROM df_dates WHERE region=='Global' AND chart=='top200'
""")
temp.createOrReplaceTempView('global_withmonth')

In [18]:
seasons = spark.sql("""
  SELECT COUNT(DISTINCT title) as title_count, artist FROM global_withmonth GROUP BY artist ORDER BY title_count DESC
""")
seasons.createOrReplaceTempView('seasons')
seasons.show()

+-----------+------------------+
|title_count|            artist|
+-----------+------------------+
|        128|      Taylor Swift|
|         95|             Drake|
|         76|               BTS|
|         72|        Juice WRLD|
|         62|            Future|
|         62|            Eminem|
|         62|        Ed Sheeran|
|         57|      Lil Uzi Vert|
|         56|     Ariana Grande|
|         49|      XXXTENTACION|
|         46|             Logic|
|         45|             Migos|
|         44|        Kanye West|
|         42|     Billie Eilish|
|         41|      Trippie Redd|
|         41|       Post Malone|
|         40|         Pop Smoke|
|         39|          Lil Baby|
|         39|     Justin Bieber|
|         38|Tyler, The Creator|
+-----------+------------------+
only showing top 20 rows



In [25]:
# winter
winter = spark.sql("""
  SELECT artist, title, month, region, rank FROM global_withmonth WHERE (month < 3) OR (month == 12)
""")
winter.createOrReplaceTempView('winterseasons')
winter2 = spark.sql("""
  SELECT COUNT(DISTINCT title) as title_count, artist FROM winterseasons GROUP BY artist ORDER BY title_count DESC
""")
winter2.createOrReplaceTempView('winterseasons')
winter3 = spark.sql("""
  SELECT title_count, artist FROM winterseasons LIMIT 10
""")
winter3 = winter3.withColumn("season", lit("winter"))
winter3.show()

+-----------+-------------+------+
|title_count|       artist|season|
+-----------+-------------+------+
|         48|       Eminem|winter|
|         39| Taylor Swift|winter|
|         38|   Juice WRLD|winter|
|         34|       Future|winter|
|         31|Ariana Grande|winter|
|         30|          BTS|winter|
|         29|    Bad Bunny|winter|
|         29|        Migos|winter|
|         28|Justin Bieber|winter|
|         28|   Ed Sheeran|winter|
+-----------+-------------+------+



In [24]:
# spring
spring = spark.sql("""
  SELECT artist, title, month, region, rank FROM global_withmonth WHERE (month > 2) AND (month < 6)
""")
spring.createOrReplaceTempView('springseasons')
spring2 = spark.sql("""
  SELECT COUNT(DISTINCT title) as title_count, artist FROM springseasons GROUP BY artist ORDER BY title_count DESC
""")
spring2.createOrReplaceTempView('springseasons')
spring3 = spark.sql("""
  SELECT title_count, artist FROM springseasons LIMIT 10
""")
spring3 = spring3.withColumn("season", lit("spring"))
spring3.show()

+-----------+------------+------+
|title_count|      artist|season|
+-----------+------------+------+
|         38|       Drake|spring|
|         38|Lil Uzi Vert|spring|
|         33|Taylor Swift|spring|
|         32|         BTS|spring|
|         31|  Ed Sheeran|spring|
|         29|  Juice WRLD|spring|
|         27|     J. Cole|spring|
|         27|  The Weeknd|spring|
|         26| Post Malone|spring|
|         24|       Logic|spring|
+-----------+------------+------+



In [26]:
# summer
summer = spark.sql("""
  SELECT artist, title, month, region, rank FROM global_withmonth WHERE (month > 5) AND (month < 9)
""")
summer.createOrReplaceTempView('summerseasons')
summer2 = spark.sql("""
  SELECT COUNT(DISTINCT title) as title_count, artist FROM summerseasons GROUP BY artist ORDER BY title_count DESC
""")
summer2.createOrReplaceTempView('summerseasons')
summer3 = spark.sql("""
  SELECT title_count, artist FROM summerseasons LIMIT 10
""")
summer3 = summer3.withColumn("season", lit("summer"))
summer3.show()

+-----------+------------------+------+
|title_count|            artist|season|
+-----------+------------------+------+
|         51|             Drake|summer|
|         42|      Taylor Swift|summer|
|         36|      XXXTENTACION|summer|
|         36|         Pop Smoke|summer|
|         34|       Linkin Park|summer|
|         32|        Kanye West|summer|
|         31|     Billie Eilish|summer|
|         28|Tyler, The Creator|summer|
|         28|        Ed Sheeran|summer|
|         28|               BTS|summer|
+-----------+------------------+------+



In [28]:
# fall
fall = spark.sql("""
  SELECT artist, title, month, region, rank FROM global_withmonth WHERE (month > 8) AND (month < 12)
""")
fall.createOrReplaceTempView('fallseasons')
fall2 = spark.sql("""
  SELECT COUNT(DISTINCT title) as title_count, artist FROM fallseasons GROUP BY artist ORDER BY title_count DESC
""")
fall2.createOrReplaceTempView('fallseasons')
fall3 = spark.sql("""
  SELECT title_count, artist FROM fallseasons LIMIT 10
""")
fall3 = fall3.withColumn("season", lit("fall"))
fall3.show()

+-----------+-------------+------+
|title_count|       artist|season|
+-----------+-------------+------+
|         59| Taylor Swift|  fall|
|         35|   Kanye West|  fall|
|         34|        Drake|  fall|
|         28|Ariana Grande|  fall|
|         27|   Ed Sheeran|  fall|
|         26|  Post Malone|  fall|
|         25|        Adele|  fall|
|         25|          BTS|  fall|
|         23|    Lil Wayne|  fall|
|         22|    Sam Smith|  fall|
+-----------+-------------+------+



In [12]:
# combining all of the seasons of top 10 into one table
all_seasons_top10 = winter3.union(spring3).union(fall3).union(summer3)

all_seasons_top10.show(40, False)

NameError: name 'winter3' is not defined

MostPopularPerYear - top 20 most popular artists per year (over the whole world, aka all regions)

In [70]:
df.createOrReplaceTempView('df1')
temp = spark.sql("""
  SELECT * FROM df1 WHERE chart='top200'
""")
temp.createOrReplaceTempView('df1')
pop_artist2017 = spark.sql("""
  SELECT artist, COUNT(*) AS SongCount FROM df1 WHERE YEAR(date) = 2017 GROUP BY artist ORDER BY SongCount DESC LIMIT 20
""")
pop_artist2018 = spark.sql("""
  SELECT artist, COUNT(*) AS SongCount FROM df1 WHERE YEAR(date) = 2018 GROUP BY artist ORDER BY SongCount DESC LIMIT 20
""")
pop_artist2019 = spark.sql("""
  SELECT artist, COUNT(*) AS SongCount FROM df1 WHERE YEAR(date) = 2019 GROUP BY artist ORDER BY SongCount DESC LIMIT 20
""")
pop_artist2020 = spark.sql("""
  SELECT artist, COUNT(*) AS SongCount FROM df1 WHERE YEAR(date) = 2020 GROUP BY artist ORDER BY SongCount DESC LIMIT 20
""")
pop_artist2021 = spark.sql("""
  SELECT artist, COUNT(*) AS SongCount FROM df1 WHERE YEAR(date) = 2021 GROUP BY artist ORDER BY SongCount DESC LIMIT 20
""")
pop_artist2017 = pop_artist2017.withColumn("year", lit("2017"))
pop_artist2018 = pop_artist2018.withColumn("year", lit("2018"))
pop_artist2019 = pop_artist2019.withColumn("year", lit("2019"))
pop_artist2020 = pop_artist2020.withColumn("year", lit("2020"))
pop_artist2021 = pop_artist2021.withColumn("year", lit("2021"))

In [71]:
popular_artists_2017to2021 = pop_artist2017.union(pop_artist2018).union(pop_artist2019).union(pop_artist2020).union(pop_artist2021)
popular_artists_2017to2021.show()

+--------------------+---------+----+
|              artist|SongCount|year|
+--------------------+---------+----+
|          Ed Sheeran|   125658|2017|
|     Imagine Dragons|    37387|2017|
|        Shawn Mendes|    33044|2017|
|          Bruno Mars|    32623|2017|
|         Major Lazer|    32021|2017|
|        Clean Bandit|    30864|2017|
|        Charlie Puth|    30670|2017|
|    The Chainsmokers|    30297|2017|
|      Kendrick Lamar|    29212|2017|
|The Weeknd, Daft ...|    27429|2017|
|               Drake|    27277|2017|
|               Ozuna|    25840|2017|
|       Justin Bieber|    23638|2017|
|Calvin Harris, Fu...|    22610|2017|
|              Maluma|    21785|2017|
|   Twenty One Pilots|    18909|2017|
|         Alan Walker|    18881|2017|
|           Nicky Jam|    18663|2017|
|        Jason Derulo|    18128|2017|
|           DJ Khaled|    18041|2017|
+--------------------+---------+----+
only showing top 20 rows



HottestSongsPerYear - hottest / top 10 trending songs (that move up the most in the charts) by year

In [22]:
df.createOrReplaceTempView('df2')
temp = spark.sql("""
  SELECT * FROM df2 WHERE chart='top200'
""")
temp.createOrReplaceTempView('df2')
hot2017 = spark.sql("""
  SELECT artist, title, COUNT(*) AS TrendCount FROM df2 WHERE trend = "MOVE_UP" AND YEAR(date) = 2017 GROUP BY artist, title ORDER BY TrendCount DESC LIMIT 10
""")
hot2018 = spark.sql("""
  SELECT artist, title, COUNT(*) AS TrendCount FROM df2 WHERE trend = "MOVE_UP" AND YEAR(date) = 2018 GROUP BY artist, title ORDER BY TrendCount DESC LIMIT 10
""")
hot2019 = spark.sql("""
  SELECT artist, title, COUNT(*) AS TrendCount FROM df2 WHERE trend = "MOVE_UP" AND YEAR(date) = 2019 GROUP BY artist, title ORDER BY TrendCount DESC LIMIT 10
""")
hot2020 = spark.sql("""
  SELECT artist, title, COUNT(*) AS TrendCount FROM df2 WHERE trend = "MOVE_UP" AND YEAR(date) = 2020 GROUP BY artist, title ORDER BY TrendCount DESC LIMIT 10
""")
hot2021 = spark.sql("""
  SELECT artist, title, COUNT(*) AS TrendCount FROM df2 WHERE trend = "MOVE_UP" AND YEAR(date) = 2021 GROUP BY artist, title ORDER BY TrendCount DESC LIMIT 10
""")
hot2017 = hot2017.withColumn("year", lit("2017"))
hot2018 = hot2018.withColumn("year", lit("2018"))
hot2019 = hot2019.withColumn("year", lit("2019"))
hot2020 = hot2020.withColumn("year", lit("2020"))
hot2021 = hot2021.withColumn("year", lit("2021"))
hot2017to2021 = hot2017.union(hot2018).union(hot2019).union(hot2020).union(hot2021)
hot2017to2021.show()

+--------------------+--------------------+----------+----+
|              artist|               title|TrendCount|year|
+--------------------+--------------------+----------+----+
|     Imagine Dragons|            Believer|      6541|2017|
|          Luis Fonsi|Despacito (Featur...|      5844|2017|
|        Clean Bandit|Rockabye (feat. S...|      5823|2017|
|  Post Malone, Quavo|     Congratulations|      5689|2017|
|          Ed Sheeran|          Photograph|      5511|2017|
|        Jason Derulo|Swalla (feat. Nic...|      5485|2017|
|Martin Garrix, Du...| Scared to Be Lonely|      5387|2017|
|          Bruno Mars|  That's What I Like|      5376|2017|
|The Chainsmokers,...|              Closer|      5368|2017|
|  ZAYN, Taylor Swift|"I Don’t Wanna Li...|      5348|2017|
|          Ed Sheeran|        Shape of You|      8043|2018|
|          Ed Sheeran|             Perfect|      7009|2018|
|            Dua Lipa|           New Rules|      6936|2018|
|            Dua Lipa|               IDG

ArtistSentiment - percentage of song sentiment per artist (using positive/negative/neutral sentiment analysis of those songs lyrics)

In [16]:
df_s.createOrReplaceTempView('sentiment')
all_regions.createOrReplaceTempView('regions')
regional = spark.sql("""
                SELECT DISTINCT artist FROM regions
""")
regional.createOrReplaceTempView('regions')
temp = spark.sql("""
                SELECT Artist, COUNT(CASE WHEN sentiment = 'Negative' THEN 1 END) AS NegativeCount, COUNT(CASE WHEN sentiment = 'Neutral' THEN 1 END) AS NeutralCount, COUNT(CASE WHEN sentiment = 'Positive' THEN 1 END) AS PositiveCount FROM sentiment GROUP BY Artist;
""")
temp.createOrReplaceTempView('sentiment')
joinem = spark.sql("""
                SELECT hs.artist, sen.NegativeCount, sen.NeutralCount, sen.PositiveCount FROM regions hs LEFT JOIN sentiment sen ON hs.artist = sen.Artist WHERE sen.Artist IS NOT NULL
""").show()

NameError: name 'all_regions' is not defined

more mysql testing / uploading tables

In [11]:
joinem.write.format("jdbc").option("driver","com.mysql.jdbc.Driver").option("url", "jdbc:mysql://localhost:3306/cs179g").option("dbtable", "ArtistSentiment").option("user", "hpham").option("password", "halpal").save()

NameError: name 'joinem' is not defined

In [48]:
db_cursor.execute("SELECT * FROM Songs;")
# Fetch all the results
results = db_cursor.fetchall()

# Display the results
for row in results:
    print(row)

In [19]:
db_cursor.execute("SHOW TABLES;")
# Fetch all the results
results = db_cursor.fetchall()

# Display the results
for row in results:
    print(row)

('ArtistByRegion',)
('ArtistSentiment',)
('HottestSongsPerYear',)
('MostPopularPerYear',)
('Seasons',)


Kanin Using for ref

In [15]:
# for year trends

db_cursor.execute("SELECT * FROM ArtistByRegion;")
results = db_cursor.fetchall()

column_names = [description[0] for description in db_cursor.description]

# # Display column names
print("Column Names:", column_names)

# Display the results
for row in results:
    print(row)

print(results)

Column Names: ['region', 'artist', 'song_count']
('Argentina', 'Maluma', 5673)
('Argentina', 'Paulo Londra', 5630)
('Argentina', 'Ed Sheeran', 5397)
('Australia', 'Ed Sheeran', 13194)
('Australia', 'Post Malone', 7907)
('Australia', 'Billie Eilish', 7141)
('Austria', 'Ed Sheeran', 5431)
('Austria', 'Capital Bra', 4695)
('Austria', 'Billie Eilish', 4275)
('Belgium', 'Ed Sheeran', 8567)
('Belgium', 'Billie Eilish', 6366)
('Belgium', 'XXXTENTACION', 5604)
('Bolivia', 'Bad Bunny', 8703)
('Bolivia', 'Maluma', 6296)
('Bolivia', 'Christian Nodal', 4676)
('Brazil', 'Marília Mendonça', 11059)
('Brazil', 'Zé Neto & Cristiano', 9385)
('Brazil', 'Henrique & Juliano', 8914)
('Bulgaria', 'Pop Smoke', 3265)
('Bulgaria', 'Travis Scott', 2760)
('Bulgaria', 'Billie Eilish', 2115)
('Canada', 'Drake', 9217)
('Canada', 'Post Malone', 8887)
('Canada', 'Ed Sheeran', 7799)
('Chile', 'Bad Bunny', 11926)
('Chile', 'Maluma', 5335)
('Chile', 'Ozuna', 5107)
('Colombia', 'Bad Bunny', 8939)
('Colombia', 'Maluma', 56

In [24]:
# for sentiments 

#db_cursor.execute("SELECT * FROM ArtistSentiment;")
db_cursor.execute("SELECT artist, NegativeCount, PositiveCount, NeutralCount FROM ArtistSentiment;")
results = db_cursor.fetchall()

# column_names = [description[0] for description in db_cursor.description]

# # Display column names
# print("Column Names:", column_names)

# Display the results
# for row in results:
#     print(row)

print(results)

[('Ozuna', 7, 5, 3), ('Maluma', 5, 3, 2), ('Capo Plaza', 1, 0, 0), ('Travis Scott', 1, 0, 0), ('Beret', 0, 0, 1), ('Ben&Ben', 0, 1, 1), ('Hov1', 4, 1, 1), ('Sech', 2, 0, 1), ('Bad Bunny', 6, 1, 0), ('KESI', 0, 2, 2), ('Lil Kleine', 1, 0, 0), ('Marília Mendonça', 0, 5, 4), ('Dua Lipa', 2, 3, 1), ('Ninho', 3, 1, 2), ('Christian Nodal', 1, 2, 0), ('PNL', 4, 4, 7), ('Damso', 2, 0, 1), ('Avicii', 0, 3, 0), ('Ultimo', 1, 1, 4), ('Justin Bieber', 0, 2, 0), ('Tulus', 0, 1, 6), ('Imagine Dragons', 1, 1, 1), ('Drake', 1, 7, 1), ('JóiPé, Króli', 1, 0, 1), ('Luis Miguel', 1, 2, 0), ('SIX60', 1, 5, 2), ('Gilli', 3, 0, 2), ('LANY', 3, 1, 0), ('Zé Neto & Cristiano', 1, 0, 0), ('Ed Sheeran', 2, 17, 1), ('Post Malone', 4, 1, 1), ('Sfera Ebbasta', 9, 1, 4), ('Wet Bed Gang', 3, 2, 0), ('J Balvin', 0, 3, 2), ('JVG', 1, 2, 10), ('Capital Bra', 2, 0, 0), ('Paulo Londra', 3, 0, 1)]
